# C# Excel tools performance comparison #

There are two tests:
- Test 1: Values - A new workbook with worksheet cell values from testData[ROWS, COLS].
- Test 2: Formatting - A new workbook with worksheet cell formatting from formatData[ROWS, COLS].

Notes: 
- Only SpreadsheetGear is tested in this notebook currently. Other .NET Excel tool products will be tested later.
- The PythonExcelToolsComparison.ipynb in this repo contains tests for multiple products


# Initialize the test data

In [1]:
using System.IO;

const int ROWS = 1000;
const int COLUMNS = 100;
const int BOLD = 1;
const int ITALIC = 2;
const int UNDERLINE = 4;
const int RED_BG = 8;

object[,] testData = new object[ROWS, COLUMNS]; 
int[,] formatData = new int[ROWS, COLUMNS];

var random = new Random();
for (int row = 0; row < ROWS; row++)
{
    for (int col = 0; col < COLUMNS; col++)
    {
        testData[row, col] = 1;
        formatData[row, col] = random.Next(0, 15);
    }
}

// Test results will be written to this output directory
string outputPath = Directory.GetCurrentDirectory();
outputPath = Path.Combine(outputPath,"output");
Console.WriteLine("Output folder path: " + outputPath);
Directory.CreateDirectory(outputPath);

Output folder path: c:\Users\Chris Hamilton\source\repos\SpreadsheetGearCodeSamples_VSCode\Samples_JupyterCSharp\output


# SpreadsheetGear tests

In [2]:
// Run this cell once to import required packages
#r "nuget:SpreadsheetGear, 9.*-*"

Installed Packages SpreadsheetGear, 9.1.19-beta

In [3]:
using SpreadsheetGear;
using System.IO;

// SpreadsheetGear License
// SpreadsheetGear for .NET Standard:  Free use is limited to 1,000 rows x 100 columns x 10 worksheets x 3 workbooks. 
// Visit https://www.spreadsheetgear.com/downloads/signedlicense.aspx for more information.
// SpreadsheetGear.Factory.SetSignedLicense("UNCOMMENT THIS LINE AND PUT YOUR SIGNED LICENSE HERE");

// Note: If you run into the SpreadsheetGear FREE use limition you can "Restart Kernel" for this Jupyter Notebook to reset it.

### Test 1: Values - SpreadsheetGear ###

In [4]:
// Populate worksheet cells with values from testData[ROWS][COLS]
var watch = System.Diagnostics.Stopwatch.StartNew();

// Note: Change LOOPNUMBER to 1 if you run into the SpreadsheetGear FREE use  
//       limitation of 1,000 rows x 100 columns x 10 worksheets x 3 workbooks
const int LOOPNUMBER = 7;

// do LOOPNUMBER times and report average time
for (int i = 0; i < LOOPNUMBER; i++)
{
    SpreadsheetGear.IWorkbook workbook = SpreadsheetGear.Factory.GetWorkbook();
    SpreadsheetGear.IWorksheet worksheet = workbook.Worksheets[0];

    SpreadsheetGear.IRange cells = worksheet.Cells;
    SpreadsheetGear.IRange range = cells[0,0,ROWS - 1,COLUMNS - 1];
    range.Value = testData;
    workbook.SaveAs(Path.Combine(outputPath,"SpreadsheetGearValues.xlsx"),SpreadsheetGear.FileFormat.OpenXMLWorkbook);
}

watch.Stop();
var elapsedMs = watch.ElapsedMilliseconds;
Console.WriteLine("Average time: " + elapsedMs/LOOPNUMBER +  " milliseconds");

Average time: 34 milliseconds


### Test 2: Formatting - SpreadsheetGear ###

In [5]:
// Populate worksheet cells with values from testData[ROWS][COLS]
var watch = System.Diagnostics.Stopwatch.StartNew();

// Note: Change LOOPNUMBER to 1 if you run into the SpreadsheetGear FREE use  
//       limitation of 1,000 rows x 100 columns x 10 worksheets x 3 workbooks
const int LOOPNUMBER = 7;

// do LOOPNUMBER times and report average time
for (int i = 0; i < LOOPNUMBER; i++)
{
    SpreadsheetGear.IWorkbook workbook = SpreadsheetGear.Factory.GetWorkbook();
    SpreadsheetGear.IWorksheet worksheet = workbook.Worksheets[0];
    SpreadsheetGear.IRange cells = worksheet.Cells;
    for (int row = 0; row < ROWS; row++)
    {
        for (int col = 0; col < COLUMNS; col++)
        {
            SpreadsheetGear.IRange cell = cells[row, col];
            cell.Value = testData[row, col];
            int formatDataValue = formatData[row, col];
            if ((formatDataValue & ITALIC) == ITALIC)
               cell.Font.Italic = true;
            if ((formatDataValue & BOLD) == BOLD)
                cell.Font.Bold = true;
            if ((formatDataValue & UNDERLINE) == UNDERLINE)
                cell.Font.Underline = SpreadsheetGear.UnderlineStyle.Single;
            if ((formatDataValue & RED_BG) == RED_BG)
                cell.Interior.Color = SpreadsheetGear.Colors.Red;
        }
    }
    workbook.SaveAs(Path.Combine(outputPath,"SpreadsheetGearFormats.xlsx"),SpreadsheetGear.FileFormat.OpenXMLWorkbook);
}

watch.Stop();
var elapsedMs = watch.ElapsedMilliseconds/LOOPNUMBER;
Console.WriteLine("Average time: " + elapsedMs + " milliseconds");


Average time: 159 milliseconds
